In [ ]:
from src import db
import yfinance as yf

db.client.conn.create_all()

with db.SessionContext() as session:
    
    for meta in db.models.Meta.query(session=session).all():
        
        source = db.models.Source.query(meta_id = meta.id).one_or_none()
        
        if source.source == "NOTSET": continue
        
        if source.source == "yahoo":
            print(meta.ticker)
            ticker = yf.Ticker(meta.ticker)
            hist = ticker.history(period="max", auto_adjust=False)
            hist.columns = hist.columns.str.replace(" ", "_")
            hist.columns = hist.columns.str.lower()
            hist.index.name = "asofdate"
            hist['open'] = hist['open'].round(2)
            hist['high'] = hist['high'].round(2)
            hist['low'] = hist['low'].round(2)
            hist['close'] = hist['close'].round(2)
            hist['tot_return'] = hist['close'].add(hist['dividends']).pct_change().fillna(0).round(5)
            hist['adj_return'] = hist['adj_close'].pct_change().fillna(0).round(5)
            hist = hist.reset_index()
            hist["meta_id"] = meta.id
            db.models.EquityDailyBar.insert(hist[["meta_id", "asofdate", "open", "high", "low", "close"]])


In [ ]:
import inspect
import pandas as pd
from src import db
from src.config import DATABASE_EXC

print(DATABASE_EXC)
# db.client.conn.create_all()

with db.SessionContext() as session:
    for name, obj in inspect.getmembers(db.models):
        
        if inspect.isclass(obj):
            print(obj)
            tablename = obj.__tablename__
            try:
                records = pd.read_excel(DATABASE_EXC, sheet_name=tablename)
            except:
                continue
            obj.update(records=records, session=session)
            print(f"update {name} complete")

        
        

In [ ]:

from src.core import metrics

import yfinance as yf

prices = yf.download("SPY, AGG")["Adj Close"]

start = metrics.to_starts(prices)
end = metrics.to_ends(prices)
end - start

In [ ]:
import glob
from datetime import date, timedelta
import numpy as np
import pandas as pd

path = "price/*.xls"
files = glob.glob(pathname=path)

yesterday = date.today() - timedelta(days=1)

for file in files:
    data = pd.read_excel(file)[["종목", "현재가"]]
    data.columns = ["ticker", "value"]
    if pd.api.types.is_object_dtype(data["value"].dtype):
        data["value"] = data["value"].str.replace(",", "").astype(float)
    data["date"] = yesterday
    data["field"] = "CLOSE"
    print(data)


In [ ]:
from src.core import strategy
from src.core import metrics
import yfinance as yf

prices = yf.download("XLU, XLK, XLB, XLP, XLY, XLI, XLV, XLF")["Adj Close"]

strategy = strategy.Momentum(prices).simulate(
    start="2015-1-1", min_volatility=0.12
)

strategy.value.plot()
print(strategy.analytics())


In [24]:
from typing import Optional
import numpy as np
import pandas as pd


class Vectorizer:
    def __init__(self, func):
        self.func = func
    
    def __call__(self, *args, **kwargs):
        print(args, kwargs)
        if 'price' in kwargs:
            if isinstance(kwargs['price'], pd.DataFrame):
                print("is dataframe")
                price = kwargs['price']
                del kwargs['price']
                print(price)
                return price.apply(self.func, **kwargs)
            elif isinstance(kwargs['price'], pd.Series):
                print("is series")
                return self.func(*args, **kwargs)
        return self.func(*args, **kwargs)

@Vectorizer
def to_start(price: pd.Series) -> pd.Timestamp:
    """calculate start date of the price"""
    return price.dropna().index[0]


import yfinance as yf

price = yf.download('SPY, AGG')["Adj Close"]
print(price)
to_start(price)


[*********************100%***********************]  2 of 2 completed
                   AGG         SPY
Date                              
1993-01-29         NaN   25.122353
1993-02-01         NaN   25.301023
1993-02-02         NaN   25.354639
1993-02-03         NaN   25.622637
1993-02-04         NaN   25.729856
...                ...         ...
2023-03-23   99.989998  393.170013
2023-03-24  100.160004  395.750000
2023-03-27   99.080002  396.489990
2023-03-28   98.930000  395.600006
2023-03-29   99.050003  401.350006

[7596 rows x 2 columns]
(                   AGG         SPY
Date                              
1993-01-29         NaN   25.122353
1993-02-01         NaN   25.301023
1993-02-02         NaN   25.354639
1993-02-03         NaN   25.622637
1993-02-04         NaN   25.729856
...                ...         ...
2023-03-23   99.989998  393.170013
2023-03-24  100.160004  395.750000
2023-03-27   99.080002  396.489990
2023-03-28   98.930000  395.600006
2023-03-29   99.050003  401.35

Timestamp('2003-09-29 00:00:00')